In [1]:
import folium
from folium import plugins
from folium.plugins import FloatImage
from folium.plugins import Fullscreen
from folium import IFrame
import pandas as pd
import base64
import sqlite3 as sql
import webbrowser


In [2]:
#create a map
my_map = folium.Map(zoom_start = 15, control_scale=True)

#add layers to the map
folium.raster_layers.TileLayer('Open Street Map').add_to(my_map)
folium.raster_layers.TileLayer('Stamen Terrain').add_to(my_map)
folium.raster_layers.TileLayer('Stamen Watercolor').add_to(my_map)
folium.raster_layers.TileLayer('Stamen Toner').add_to(my_map)

#add a layer control option
folium.LayerControl().add_to(my_map)

#add fullscreen option to the map
Fullscreen(position='topleft', title='Full Screen', title_cancel='Exit Full Screen', force_separate_button=False).add_to(my_map)

In [3]:
#List of the different files containing species informations
Animals = ["Frogs.csv", "Snakes.csv"] 

#executes for all files in 'Animals' list
for i in range(len(Animals)):
    
#open file using pandas
    Animal = pd.read_csv(Animals[i])

#will create a marker for each row of the previously opened file
    for _, animal in Animal.iterrows():   
                jpg=animal['Image']   #set the content of the 'Image' column in 'jpg' variable
                encoded = base64.b64encode(open(jpg, 'rb').read())   #opens the image file using 'jpg' variable and makes it usable
                svg = """
                <object data="data:image/jpg;base64,{}" width="{}" height="{} type="image/svg+xml">
                </object>""".format   #store the image in 'svg' variable using html
                #stores in 'html' variable a set of informations from the previously opened file, of the current row using html
                html = """
                <h1>{Name}</h1><p>Discovery date : {DYear}<br>Family : {Fam}<br>
                <a href="https://en.wikipedia.org/wiki/{Wiki}">
                More Informations<br><br></a>""".format( Name=animal['Name'], DYear=animal['Discovery Year'], Fam=animal['Family'], Wiki=animal['Wiki'])
                html = html + svg(encoded.decode('UTF-8'), 250, 170)  #combines the informations and image into one variable
                iframe = IFrame(html=html, width=300, height=200)
                popup = folium.Popup(iframe, parse_html = True, max_width=1550) #sets up the popup that will be added
        
                folium.Marker(location=[animal['Latitude'], animal['Longitude']],popup = popup,          
                              icon= folium.Icon(color=animal['Status'], icon_color='white',icon = 'globe')
                              ).add_to(my_map)   #adds the marker to the map
            
my_map.save("map.html")   #saves the map

In [4]:
#Creates a legend for the map using html
lgd = """<br> <span style="color: {col};">{item}</span> &nbsp; <i class="
glyphicon glyphicon-globe" style="color:{col}"></i> </li>"""
            
html_itms = lgd.format(item="Least Concerned", col="#1BB5E0") \
            + lgd.format(item="Near Threatened", col="#19B015") \
            + lgd.format(item="Slightly Threatened", col="#FD9E98") \
            + lgd.format(item="Threatened", col="orange") \
            + lgd.format(item="Highly Threatened", col="red") \
            + lgd.format(item="Extinct", col="black") \

legend_html = """
     <div style="
     position: fixed; 
     top: 200px; left: 0px; width: 20x; height: 90x; 
     border:5px solid grey; 
     border-radius: 5px;
     z-index:9999; 

     background-color:white;     opacity: .99;
     <!-- background-color:rgba(255, 255, 255, 0.99); -->
     border-radius:6px;

     font-size:13px;
     font-weight: bold;

     "> {itm_txt}
     </ul>
     </div> """.format(itm_txt= html_itms)

#adds the legend to the map
my_map.get_root().html.add_child(folium.Element( legend_html ))


In [5]:
#Adds markers to the map depending on the data already contained in the database
def UpdateMap(animaldata):
    connection = sql.connect('Database.db')   #connection to the database
    query = "SELECT * FROM " + animaldata    #selects all elements in 'animaldata' variable
    df = pd.read_sql_query(query, connection)   #using pandas to read the sql query and stores it in 'df' variable
    df.to_csv('test.csv')   #puts the content of 'df' into a csv file and naming it 'test.csv'
    data = pd.read_csv('test.csv')   #reads the csv file using pandas
    
    #adds a marker to the map for all info contained into the database
    for _, animal in data.iterrows():
                    html = """
                    <h1>{Name}</h1><p>Discovery date : {DYear}<br>Family : {Family}<br>
                    <a href="https://en.wikipedia.org/wiki/{Wiki}">
                    More Informations<br><br></a>""".format( Name=animal['Name'], DYear=animal['Discovery Year'], Family=animal['Family'], Wiki=animal['Wiki'])
                    iframe = IFrame(html=html, width=300, height=200)
                    popup = folium.Popup(iframe, parse_html = True, max_width=1550)
        
                    folium.Marker(location=[animal['Latitude'], animal['Longitude']],popup = popup,          
                                  icon= folium.Icon(color=animal['Endangerment Level'], icon_color='white',icon = 'globe')
                                  ).add_to(my_map) 

    my_map.save("map.html")

In [6]:
#sets to 'database' variable the name of the database file
database = "Database.db"
connection = sql.connect(database)   #connects to the data base
cur = connection.cursor()   #creates a cursor
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")   #Selects names of all tables in the database
row = cur.fetchone()   #fetch one tabel name

#Will call the previously created function 'UpdateMap' for all tables in the database
while row is not None:   
    UpdateMap(''.join(row))   #using 'join(row)' because the name of the table isnt a string
    row = cur.fetchone()
    
cur.close()   #closes the cursor
connection.close()   #closes the connection

In [7]:

while(1):
    #asks the user to chose an action
    print("\nEnter the number corresponding to the desired action: \n1 - View Map\n2 - View Data\n3 - Add Data\n4 - Delete Data\n5 - Update Data\n6 - Create Table\n7 - Delete Table\nAnything else - Exit")
    choice = input()   #stores his answer in 'choice' variable

    #will do the desired action depending on 'choice' 
    
    if(choice == '1'):   #View Map
        
        #opens the map in a webbrowser
        webbrowser.open("map.html")

    elif(choice == '2'):   #View data
        database = "Database.db"
        connection = sql.connect(database)
        cur = connection.cursor()
        cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
        print(cur.fetchall())   #prints the names of all table names
        print("What animal list do you want to interact with: ")
        animaldata = input()
        query = "SELECT * FROM " + animaldata   
        df = pd.read_sql_query(query, connection)
        print(df.head())   #prints the informations of all species in the selected table
        cur.close()
        connection.close()
    
    elif(choice == '4'):   #Delete data
        database = "Database.db"
        connection = sql.connect(database)
        cur = connection.cursor()
        print("Enter the name of the species you wish to delete: ")
        DName = input()
        cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
        row = cur.fetchone()
        
        while row is not None:   #it will delete the species in all tables
            query = "DELETE FROM "+''.join(row)+" WHERE Name = (?)"   
            cur.execute(query, (DName,))
            connection.commit()
            row = cur.fetchone()
        
        cur.close()
        connection.close()
        print("\nThe data has been deleted successfully !\n")
        
    elif(choice == '5'):   #Modify data
        database = "Database.db"
        connection = sql.connect(database)
        cur = connection.cursor()
        print("Enter the name of the species you wish to update the data from: ")
        UName = input()
        cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
        print(cur.fetchall())
        print("Which list contains the species: ")
        animalName=input()
        
        #asks the user which column data he wants to modify
        print("\nName | Family | Discovery Year | Endangerment Level | Latitude | Longitude | Wiki\nEnter the column you wish to update: ")
        Col = input()
        print("Enter the new data: ")
        New = input()
        query='UPDATE '+animalName+' SET "'+Col+'" = '+New+' WHERE Name = ?'   #inserts the variables into the query
        cur.execute(query, (UName,))
        connection.commit()
        UpdateMap(animalName)
        cur.close()
        connection.close()
        print("\nThe data has been updated successfully !\n")
         
    elif(choice == '3'):   #View data
        database = "Database.db"
        connection = sql.connect(database)
        cur = connection.cursor()
        cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
        print(cur.fetchall())
        print("What animal list do you want to interact with: ")
        animalName = input()
        
        #Asks the user to enter all the informations of the species he wants to add and stores them into variables 
        print("Enter the name of the species: ")
        name = input()
        print("Enter the name of the family of the species: ")
        family = input()
        print("Enter the year of discovery: ")
        year = input()
        print("\nLeast Concerned : blue\nNear Threatened : green\nSlightly Threatened : pink\nThreatened : orange\nHighly Threatened : red\nExtinct : black\nEnter the colour depending on the endangerment of the species: ")
        color = input()
        print("Enter the latitude of the location where the species is found: ")
        latitude = input()
        print("Enter the longitude of the location where the species if found: ")
        longitude = input()
        print("Enter the name of the Wikipedia page of the species (usually adding '_' in the name if there is a space is sufficient): ")
        wiki = input()
    
        query = "INSERT INTO "+animalName+" VALUES (?, ?, ?, ?, ?, ?, ?)"   #Creates the query for inserting data
        cur.execute(query, (name, family, year, color, latitude, longitude, wiki))   #Adds the variable to the query
        connection.commit()   
        UpdateMap(animalName)
        cur.close
        connection.close()
        print("\nThe data has been added successfully !\n")
        
    elif(choice == '6'):   #Create a table
        database = "Database.db"
        connection = sql.connect(database)
        print("Enter the name of the table you want to create: ")
        name = input()
        #Creates the table
        query = 'CREATE TABLE ' +name+ '("Name" TEXT NOT NULL UNIQUE,"Family" TEXT,"Discovery Year" INTEGER,"Endangerment Level" TEXT,"Latitude" NUMERIC,"Longitude" NUMERIC,"Wiki" TEXT,PRIMARY KEY("Name"));'
        cur = connection.cursor()
        cur.execute(query)
        connection.commit()
        cur.close()
        connection.close()
        print("Table "+name+" created successfully !")
        
    elif(choice == '7'):   #Delete table
        database = "Database.db"
        connection = sql.connect(database)
        cur = connection.cursor()
        cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
        print(cur.fetchall())
        print("\nWhich table would you like to delete: ")
        name = input()
        query = 'DROP TABLE '+name   #deletes the table
        cur.execute(query)
        connection.commit()
        cur.close()
        connection.close()
        print("Table "+name+" deleted successfully !")
    
    #If the user enters somethings that dont have a specific effect, the program will close
    else:  
        break




Enter the number corresponding to the desired action: 
1 - View Map
2 - View Data
3 - Add Data
4 - Delete Data
5 - Update Data
6 - Create Table
7 - Delete Table
Anything else - Exit
2
[('Frog',), ('Snake',)]
What animal list do you want to interact with: 
Frog
                     Name         Family  Discovery Year Endangerment Level  \
0  Dendrobates tinctorius  Dendrobatidae            1861               blue   

   Latitude  Longitude                    Wiki  
0      6.66     -74.54  Dendrobates_tinctorius  

Enter the number corresponding to the desired action: 
1 - View Map
2 - View Data
3 - Add Data
4 - Delete Data
5 - Update Data
6 - Create Table
7 - Delete Table
Anything else - Exit
4
Enter the name of the species you wish to delete: 
Dendrobates tinctorius

The data has been deleted successfully !


Enter the number corresponding to the desired action: 
1 - View Map
2 - View Data
3 - Add Data
4 - Delete Data
5 - Update Data
6 - Create Table
7 - Delete Table
Anything else - E